In [4]:
import os, ast
from os import listdir
from os.path import isfile, join

import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error

In [5]:
######################## AUDIO TRAINING #####################
#audio files include IEMOCAP data, Toronto Emotion Speech Dataset and IHMC Data

#read audio data features (csv file) from opensmile processing
with open('IEMOCAPvectorsfinal.txt') as t:
    audioIEMOCAP_features = [map(float,line.split()) for line in t]

for i in range(0,len(audioIEMOCAP_features)):
    audioIEMOCAP_features[i] = list(audioIEMOCAP_features[i])

#other audio: Toronto & IHMC
with open('otheraudiovectors.txt') as t:
    other_audio_features = [map(float,line.split()) for line in t]

for j in range(0,len(other_audio_features)):
    other_audio_features[j] = list(other_audio_features[j])

#read emotional labels from file
other_audio_labels = np.load('otheraudiolabelsfixed.npy')

#read filenames that has 
with open("IEMOCAP_filenames.txt") as f:
    fnames_with_feat = f.read().splitlines() 

#reading the labels from label file
line_dict = open("IEMOCAP_label_dict2.txt").read()

line_dict = ast.literal_eval(line_dict)
IEMOCAPlabels = []            
for i in fnames_with_feat:
    IEMOCAPlabels.append(line_dict[i])

In [6]:
#to remove all emotions that are not one of the following, find their indices and store them
takeouts = []
for i in range(0,len(IEMOCAPlabels)):
    if IEMOCAPlabels[i] != ['Happiness'] and IEMOCAPlabels[i] != ['Sadness'] and IEMOCAPlabels[i] != ['Fear'] and IEMOCAPlabels[i] != ['Anger']: 
        takeouts.append(i)

In [7]:
#remove unneeded labels/output info for the indices
tokeeplabels = []
for i in range(0,len(IEMOCAPlabels)):
    if i not in takeouts:
        tokeeplabels.append(IEMOCAPlabels[i])

In [8]:
#only keep samples/vectors of information with wanted emotions (happiness, joy, fear, sadness)
tokeepsamples = []
for i in range(0,len(audioIEMOCAP_features)):
    if i not in takeouts:
        tokeepsamples.append(audioIEMOCAP_features[i])

In [9]:
#some data preprocessing
for i in range(0,len(tokeeplabels)):
    if tokeeplabels[i] == ['Happiness']:
        tokeeplabels[i] = 'joy'
    if tokeeplabels[i] == ['Anger']:
        tokeeplabels[i] = 'anger'
    if tokeeplabels[i] == ['Fear']:
        tokeeplabels[i] = 'fear'
    if tokeeplabels[i] == ['Sadness']:
        tokeeplabels[i] = 'sadness'

In [10]:
X_train, X_test, y_train, y_test = train_test_split(tokeepsamples, tokeeplabels, test_size=0.2, random_state=42)
#X_test and y_test to be used for the actual test section later
#X_train and y_train to be added to other audio features and labels

In [11]:
#joining audio features from both samples (they're all of vector length ~1500)
audiofeatures = []
for i in range(0,len(X_train)):
    audiofeatures.append(X_train[i])

for j in range(0,len(other_audio_features)):
    audiofeatures.append(other_audio_features[j])

audiofeatures = np.asarray(audiofeatures)

#need to add 80% of IEMOCAP to other audio to get 
labels = np.append(y_train,other_audio_labels) #this append only works bc the labels are all single strings

In [12]:
#SVM classifier

#loading audio classifier
audioclf = svm.SVC(class_weight='balanced')
#AUDIO TRAINING: Toronto Data (100%), IHMC Berlin Data (100%), IEMOCAP Data (80%)
audioclf.fit(audiofeatures,labels)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
#we now use transcriptions of part of the audio we have for our text model
with open('IEMOCAP_transcriptions2.txt') as f:
    IEMOCAPtext = f.read().splitlines() 

In [14]:
#keep the transcriptions only of the emotions we want
tokeepsentences= []
for i in range(0,len(IEMOCAPtext)):
    if i not in takeouts:
        tokeepsentences.append(IEMOCAPtext[i])

In [15]:
#gets ontology vectors
with open('IEMOCAP_ont_vecs.txt') as t:
    IEMOCAP_ont_vecs = [map(float,line.split()) for line in t]
for i in range(0,len(IEMOCAP_ont_vecs)):
    IEMOCAP_ont_vecs[i] = list(IEMOCAP_ont_vecs[i])
IEMOCAP_ont_vecs = np.asarray(IEMOCAP_ont_vecs)

# gets w2v vectors
with open('avgIEMOCAPfinalw2v.txt') as t:
    IEMOCAP_w2v = [map(float,line.split()) for line in t]
for i in range(0,len(IEMOCAP_w2v)):
    IEMOCAP_w2v[i] = list(IEMOCAP_w2v[i])
IEMOCAP_w2v = np.asarray(IEMOCAP_w2v)

# averaging the two 
final_IEMOCAP_txt_vec = []
for v in range(len(IEMOCAP_w2v)):
    if np.count_nonzero(IEMOCAP_ont_vecs[v]) == 300:
        final_IEMOCAP_txt_vec.append(IEMOCAP_ont_vecs[v])
    else:
        temp_vec = (IEMOCAP_ont_vecs[v] + IEMOCAP_w2v[v]) / 2
        final_IEMOCAP_txt_vec.append(temp_vec)

In [16]:
#splitting IEMOCAP data specificially, because it must be used for test portion
X_trainItxt, X_testItxt, y_trainItxt, y_testItxt = train_test_split(final_IEMOCAP_txt_vec, tokeeplabels, test_size=0.2, random_state=42)

In [17]:
#also train NLP model with twitter emotion labeled data, SemEval Competition 2018
with open('twitter_vecs.txt') as t:
    twitter_vecs = [map(float,line.split()) for line in t]
for i in range(0,len(twitter_vecs)):
    twitter_vecs[i] = list(twitter_vecs[i])

In [19]:
import csv 
#######TEXT TRAINING: Tweets (100%), IEMOCAP Data (80%) - PER EMOTION!!!!#############
textfeatures = []

#join together the two data types, features are vectors of length 300 w2v and ontology averages
for j in range(0,len(X_trainItxt)): #IEMOCAP 80%
    textfeatures.append(X_trainItxt[j])
    
for k in range(0,len(twitter_vecs)): #tweets
    textfeatures.append(twitter_vecs[k])

#open emotion labels and format
tweetlabels = []
with open('twitter_labels.txt') as inputfile:
    for row in csv.reader(inputfile):
        tweetlabels.append(row)
for i in range(0,len(tweetlabels)):
    tweetlabels[i] = tweetlabels[i][0]

In [20]:
#all tweet labels
addonlabels=[]
with open('alllabels.txt') as inputfile:
    for row in csv.reader(inputfile):
        addonlabels.append(row)

In [21]:
#preprocessing due to file glitch
for i in range(0,len(addonlabels)):
    if addonlabels[i] == ['a\tn\tg\te\tr']:
        tweetlabels.append('anger')
    elif addonlabels[i] == ['f\te\ta\tr']:
        tweetlabels.append('fear')
    elif addonlabels[i] == ['j\to\ty']:
        tweetlabels.append('joy')
    else:
        tweetlabels.append('sadness')
tweetlabels.append('sadness')

In [22]:
#need to add 80% of IEMOCAP to other audio to get 
textlabels = np.append(y_trainItxt,tweetlabels) #this append only works bc the labels are all single strings

In [23]:
#additional data we only have w2v for (no ontology)
w2v = np.loadtxt('w2vtweets.txt')

In [24]:
for i in range(0,len(w2v)):
    textfeatures.append(w2v[i])
textfeatures = np.asarray(textfeatures)

In [25]:
#loading text classifier
textclf = svm.SVC(class_weight='balanced')
textclf.fit(textfeatures,textlabels)
######################################################################################

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
#format all data needed for regression portion
##################### ANGER ############################

with open('averageanger.txt') as t:
    XA_train = [map(float,line.split()) for line in t]

for i in range(0,len(XA_train)):
    XA_train[i] = list(XA_train[i])
XA_train = np.asarray(XA_train)

with open('angerscores.txt') as t:
    yatrain = [map(float,line.split()) for line in t]

for i in range(0,len(yatrain)):
    yatrain[i] = list(yatrain[i])
yatrain = np.asarray(yatrain)

####################   FEAR    ############################

with open('averagefear.txt') as t:
    XF_train = [map(float,line.split()) for line in t]

for i in range(0,len(XF_train)):
    XF_train[i] = list(XF_train[i])
XF_train = np.asarray(XF_train)

with open('fearscores.txt') as t:
    yftrain = [map(float,line.split()) for line in t]

for i in range(0,len(yftrain)):
    yftrain[i] = list(yftrain[i])
yftrain = np.asarray(yftrain)

####################   JOY    #############################

with open('averagejoy.txt') as t:
    XJ_train = [map(float,line.split()) for line in t]

for i in range(0,len(XJ_train)):
    XJ_train[i] = list(XJ_train[i])
XJ_train = np.asarray(XJ_train)

with open('joyscores.txt') as t:
    yjtrain = [map(float,line.split()) for line in t]

for i in range(0,len(yjtrain)):
    yjtrain[i] = list(yjtrain[i])
yjtrain = np.asarray(yjtrain)

#################### SADNESS ##############################

with open('averagesadness.txt') as t:
    XS_train = [map(float,line.split()) for line in t]

for i in range(0,len(XS_train)):
    XS_train[i] = list(XS_train[i])
XS_train = np.asarray(XS_train)

with open('sadnesscores.txt') as t:
    ystrain = [map(float,line.split()) for line in t]

for i in range(0,len(ystrain)):
    ystrain[i] = list(ystrain[i])
ystrain = np.asarray(ystrain)
###############################################################

In [28]:
#################    REGRESSION TRAINING  #####################
#anger split, model, and fitting
angerregression = svm.SVR()
angerregression.fit(XA_train,yatrain.ravel()) 

#fear split, model, and fitting
fearregression = svm.SVR()
fearregression.fit(XF_train,yftrain.ravel()) 

#joy split, model, and fitting
joyregression = svm.SVR()
joyregression.fit(XJ_train,yjtrain.ravel()) 

#sadness model, and fitting
sadnessregression = svm.SVR()
sadnessregression.fit(XS_train,ystrain.ravel())

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [29]:
######################### TEST PHASE ####################################
tpredicted = textclf.predict(X_testItxt)
apredicted = audioclf.predict(X_test)

In [30]:
agreed = 0
for i in range(0,len(tpredicted)):
    if tpredicted[i] == apredicted[i]: #only one regression model needed, they agreed!
        agreed +=1
        if tpredicted[i] == 'anger':
            intensity = angerregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'fear':
            intensity = fearregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'joy':
            intensity = joyregression.predict(X_testItxt[i].reshape(1, -1))
        else:
            intensity = sadnessregression.predict(X_testItxt[i].reshape(1, -1))
            
        print("This sample was found to be " + tpredicted[i] + " with intensity " + str(intensity))
    else: #they gave different emotions, need to do both
        if tpredicted[i] == 'anger':
            intensity1 = angerregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'fear':
            intensity1 = fearregression.predict(X_testItxt[i].reshape(1, -1))
        elif tpredicted[i] == 'joy':
            intensity1 = joyregression.predict(X_testItxt[i].reshape(1, -1))
        else:
            intensity1 = sadnessregression.predict(X_testItxt[i].reshape(1, -1))
            
        if apredicted[i] == 'anger':
            intensity2 = angerregression.predict(X_testItxt[i].reshape(1, -1))
        elif apredicted[i] == 'fear':
            intensity2 = fearregression.predict(X_testItxt[i].reshape(1, -1))
        elif apredicted[i] == 'joy':
            intensity2 = joyregression.predict(X_testItxt[i].reshape(1, -1))
        else:
            intensity2 = sadnessregression.predict(X_testItxt[i].reshape(1, -1))
            
        print("This sample was found have " + tpredicted[i] + " or " + apredicted[i] + " with intensity " + str(intensity1) + ", " + str(intensity2) + " respectively.")

This sample was found have sadness or anger with intensity [0.50447284], [0.50210564] respectively.
This sample was found have sadness or anger with intensity [0.50645316], [0.50543008] respectively.
This sample was found have sadness or anger with intensity [0.50803195], [0.50859048] respectively.
This sample was found to be sadness with intensity [0.50374607]
This sample was found have sadness or anger with intensity [0.50484636], [0.50772016] respectively.
This sample was found have fear or joy with intensity [0.4961195], [0.4998168] respectively.
This sample was found have fear or anger with intensity [0.49747017], [0.49943695] respectively.
This sample was found have sadness or anger with intensity [0.50627339], [0.50464942] respectively.
This sample was found have fear or anger with intensity [0.49593153], [0.49703734] respectively.
This sample was found have sadness or anger with intensity [0.50617511], [0.50533704] respectively.
This sample was found have fear or anger with int

This sample was found have fear or anger with intensity [0.49570675], [0.49581368] respectively.
This sample was found have fear or anger with intensity [0.49612188], [0.49725449] respectively.
This sample was found have fear or anger with intensity [0.49585261], [0.49635322] respectively.
This sample was found have sadness or anger with intensity [0.50651925], [0.50305951] respectively.
This sample was found have fear or sadness with intensity [0.49577212], [0.50367099] respectively.
This sample was found have fear or anger with intensity [0.49574332], [0.49617142] respectively.
This sample was found have fear or anger with intensity [0.49575934], [0.49589962] respectively.
This sample was found have fear or anger with intensity [0.49572317], [0.49587317] respectively.
This sample was found have fear or joy with intensity [0.49599016], [0.49983233] respectively.
This sample was found have fear or anger with intensity [0.49569867], [0.49589351] respectively.
This sample was found have 

This sample was found have sadness or anger with intensity [0.50487585], [0.50480361] respectively.
This sample was found have fear or sadness with intensity [0.49576638], [0.50373733] respectively.
This sample was found have sadness or anger with intensity [0.50528342], [0.50612368] respectively.
This sample was found have sadness or anger with intensity [0.50820082], [0.50863108] respectively.
This sample was found have sadness or anger with intensity [0.50517242], [0.50393707] respectively.
This sample was found have fear or anger with intensity [0.49587304], [0.49662025] respectively.
This sample was found have fear or anger with intensity [0.49575291], [0.49619567] respectively.
This sample was found have fear or anger with intensity [0.49571993], [0.49584698] respectively.
This sample was found have fear or anger with intensity [0.49578207], [0.49608391] respectively.
This sample was found have sadness or anger with intensity [0.50225228], [0.50216423] respectively.
This sample w

This sample was found to be sadness with intensity [0.50284794]
This sample was found have fear or anger with intensity [0.49580601], [0.49622346] respectively.
This sample was found have sadness or anger with intensity [0.50531937], [0.50443911] respectively.
This sample was found have fear or anger with intensity [0.49592105], [0.49669261] respectively.
This sample was found have fear or anger with intensity [0.49574561], [0.49605085] respectively.
This sample was found have sadness or anger with intensity [0.50488681], [0.5053401] respectively.
This sample was found have sadness or anger with intensity [0.50643692], [0.50539703] respectively.
This sample was found have sadness or anger with intensity [0.50382084], [0.50294669] respectively.
This sample was found to be sadness with intensity [0.50465616]
This sample was found to be sadness with intensity [0.5092118]
This sample was found have sadness or anger with intensity [0.50500792], [0.50357835] respectively.
This sample was fou

This sample was found have fear or anger with intensity [0.49579198], [0.49643008] respectively.
This sample was found have sadness or anger with intensity [0.50606983], [0.50377249] respectively.
This sample was found have sadness or anger with intensity [0.50359455], [0.49798779] respectively.
This sample was found have sadness or anger with intensity [0.50549621], [0.50478091] respectively.
This sample was found have sadness or anger with intensity [0.50101811], [0.50202007] respectively.
This sample was found have sadness or anger with intensity [0.50323001], [0.50393814] respectively.
This sample was found have sadness or anger with intensity [0.5029613], [0.50033153] respectively.
This sample was found to be sadness with intensity [0.50249517]
This sample was found have sadness or anger with intensity [0.50506642], [0.50469666] respectively.
This sample was found have sadness or anger with intensity [0.50500576], [0.50420755] respectively.
This sample was found have sadness or an

This sample was found have sadness or anger with intensity [0.50802774], [0.50988604] respectively.
This sample was found have fear or anger with intensity [0.49581371], [0.49613417] respectively.
This sample was found have sadness or anger with intensity [0.50716402], [0.50705976] respectively.
This sample was found have sadness or anger with intensity [0.50582255], [0.50332648] respectively.
This sample was found have fear or anger with intensity [0.49575906], [0.4960429] respectively.
This sample was found have sadness or anger with intensity [0.50397772], [0.50286584] respectively.
This sample was found to be anger with intensity [0.50347775]
This sample was found have sadness or anger with intensity [0.50498082], [0.50390194] respectively.
This sample was found have sadness or anger with intensity [0.50424335], [0.50329353] respectively.
This sample was found have sadness or joy with intensity [0.50700918], [0.49988732] respectively.
This sample was found have fear or anger with i

In [31]:
agreed/len(tpredicted)

0.08006535947712418

In [32]:
from sklearn.metrics import accuracy_score
print(accuracy_score(tpredicted,y_test))
print(accuracy_score(apredicted,y_test))

0.184640522875817
0.5473856209150327


In [33]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, tpredicted))
print(confusion_matrix(y_test, apredicted))

[[  5  90   0 143]
 [  1  10   0   6]
 [  3  66   0  82]
 [  4 104   0  98]]
[[238   0   0   0]
 [ 17   0   0   0]
 [129   0  22   0]
 [131   0   0  75]]
